Defines a secure function (get_account_by_id_safe) that retrieves records using a parameterized query.

Defines an insecure function (get_account_by_id_insecure) that demonstrates SQL injection risks.

Executes both functions, showing the secure approach and the risk of SQL injection.

Expected Results:
Safe Query: Returns a single row matching the provided id.

Insecure Query: Executes unintended SQL, returning all accounts (demonstrating an injection risk).

In [1]:
import sqlite3
import pandas as pd

# Connect to SQLite database (or create it)
database_path = "bank.db"
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

# Create accounts table
cursor.execute("""
CREATE TABLE IF NOT EXISTS accounts (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    balance INTEGER
);
""")

# Insert sample data
cursor.executemany("""
INSERT INTO accounts (name, balance) VALUES (?, ?);
""", [('Alice', 10), ('Bob', 20), ('Charlie', 30)])

conn.commit()  # Save changes

# Function to fetch data securely using parameterized queries
def get_account_by_id_safe(account_id):
    query = "SELECT * FROM accounts WHERE id = ?"
    df = pd.read_sql_query(query, conn, params=(account_id,))
    return df

# Function to demonstrate SQL injection risk
def get_account_by_id_insecure(account_id):
    query = f"SELECT * FROM accounts WHERE id = {account_id}"
    print("Executing:", query)
    df = pd.read_sql_query(query, conn)
    return df

# Secure query example
print("Safe Query Output:")
display(get_account_by_id_safe(1))

# Insecure query example (simulating SQL injection)
print("Insecure Query Output:")
display(get_account_by_id_insecure("1 UNION SELECT * FROM accounts"))

# Close connection
conn.close()


Safe Query Output:


,id,name,balance
0,1,Alice,10


Insecure Query Output:
Executing: SELECT * FROM accounts WHERE id = 1 UNION SELECT * FROM accounts


,id,name,balance
0,1,Alice,10
1,2,Bob,20
2,3,Charlie,30
